In [1]:
%pwd

'/home/techie/Desktop/general/end-to-end-kidney-tumor/research'

In [2]:
import os

os.chdir('../')
%pwd

'/home/techie/Desktop/general/end-to-end-kidney-tumor'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/fraidoon_omarzai/end-to-end-kidney-tumor.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="fraidoon_omarzai"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bc25b16bd5206328d8899cf34377f26ad71d1420"

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [11]:
from kidney.constants import *
from kidney.utils.common import read_yaml, save_json

In [16]:
class ConfigurationManager:
    def __init__(self, config=CONFIG_PATH, params=PARAMS_PATH):
        self.config = read_yaml(config)
        self.params = read_yaml(params)
        
    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        model_evaluation_config = ModelEvaluationConfig(
            path_of_model=config.path_of_model,
            training_data=config.training_data,
            mlflow_uri=config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return model_evaluation_config

In [17]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2023-11-17 17:48:52,433: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-17 17:48:52,443: INFO: common: yaml file: params.yaml loaded successfully]


Found 139 images belonging to 2 classes.


2023-11-17 17:48:59.130479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-11-17 17:49:01.559613: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-17 17:49:03.585231: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-17 17:49:07.163143: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-11-17 17:49:07.607961: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-11-17 17:49:09.177975: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 1

5/5 [==============================] - 167s 33s/step - loss: 27.6865 - accuracy: 0.4820
[2023-11-17 17:51:46,448: INFO: common: json file saved at: scores.json]


2023/11/17 17:51:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-11-17 17:52:01,685: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2023-11-17 17:52:04,671: INFO: builder_impl: Assets written to: /tmp/tmpu2v4byut/model/data/model/assets]


2023/11/17 17:54:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpu2v4byut/model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
/home/techie/anaconda3/envs/kidney-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2023-11-17 17:59:19,110: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'timeout('The write operation timed out')': /fraidoon_omarzai/end-to-end-kidney-tumor.mlflow/api/2.0/mlflow-artifacts/artifacts/d4c4564f634040ea88fe8090ca5f1f17/ecaccc08ad2c425585b2c5caad408518/artifacts/model/data/model/variables/variables.data-00000-of-00001]


KeyboardInterrupt: 